In [2]:
import hashlib
from typing import Iterable, Tuple, List, Dict, Set
from functools import lru_cache
from collections import Counter, defaultdict

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
import IPython.display as ipd
import nltk
from tqdm.auto import tqdm
from crowdkit.aggregation import ROVER

In [3]:
train_df = pd.read_json("data/noisy_text_aggregation_train.jsonl", lines=True)

train_df.head()

,task,text,qnet,w2v,w2v_tts
0,ed0f6706f75681a7915fec15d336aca5,дамира николаевича,дамира николаевича,до мира николаевича,домера николаевича
1,3c2875271fb918da312865549d444653,антонине татариновой,онпанине то тариновый,антонине татариновый,антонини татариновый
2,f3dba379c6280536aaa65a56c4358268,жидкову,жит куву,жит куву,жидкову
3,98962310d56cd7095d9893f5ed657f81,главатских,лалаки,главатских,главацки
4,b4f6a8d9e2eba8085d25d4122b52d55a,мошкова,мошкова,машкова,мошкова


In [4]:
test_df = pd.read_json("data/noisy_text_aggregation_test.jsonl", lines=True)

test_df.head()

,task,qnet,w2v,w2v_tts
0,73aff8bc8d99278c6ca6d1ac243557e3,дели,зили,зили
1,ba8443f3cc91e56667278db36dea02b7,вектор упавнович чунула к лещенуму,виктору павловичу новокрещеному,виктору павлновичу навокрещинову оо
2,af346fae1d5686a357e32710c5d4f13e,риме менниковой,римми мельниковой,рымми мельниковой
3,262fda7ab40a8417b99ecb314b3f7405,александровик провичо,александру викторовичу,александру викторовичу
4,08bd8fb35ceaf2843734ffeb389f2283,я ро славцов,ерославцев,ярославцев


In [5]:
text_data = pd.read_csv("data/noisy_text_aggregation_text_only (1).csv")

text_data.head(10)

,алиби
0,сказка
1,найди в нете мульт стальной гигант
2,мона лиза
3,смотреть сериал восемь с половиной
4,вруби российский фильм мосгорсмех
5,найди на ютубе фильм меня зовут мохаммед али
6,можешь мне поставить мультфильм квадратные зве...
7,сваты побыстрее включи пожалуйста
8,рокетмен смотреть сериал рокетмен
9,можешь поставить мировая прогулка индия гоа


In [6]:
MODEL_LIST = ["qnet", "w2v", "w2v_tts"]

In [7]:
def hash_reminder(str_, base: int=10) -> int:
    return int(hashlib.md5(str_.encode()).hexdigest(), 16) % base

train_mask = train_df['task'].apply(lambda x: hash_reminder(x, 10) <= 7)

val_df = train_df[~train_mask]
train_df = train_df[train_mask]
val_df

,task,text,qnet,w2v,w2v_tts
1,3c2875271fb918da312865549d444653,антонине татариновой,онпанине то тариновый,антонине татариновый,антонини татариновый
6,1254c97d2434297a886e4dc4b2a16863,илюши власова,илюши влассво,и люши власово,и люше власова
7,867e45b9a914fffff0c8ef976ee0a275,терехину,тере гину,терехину,терьехину
19,32d052f80c431869e7c13914b88f958e,айдарчике,а и дарчики,аидарчике,а и дарчике
21,719cbc316c0f3caa6324c110ad899b81,ксении валерьевны волковой,сени и воверивны в волколы,к сени и валеревны волковой,ксение ивалиревной волковой
...,...,...,...,...,...
60901,b2c4360bffc98f20b59fc6e6c3f1a606,фармэконом,фар мы коном,фар мконом,фар мыконон
60907,de0fd646bdd3b37697580c2cc2d7d624,ссср бронь,этозысер бронь,эсососэр бронь,эсисыцар бронь
60909,97d2af838eb539e5a7151b099e13498d,фото салончик,хот то солоничиак,фот -то солончик,фото солоничек а
60911,f8df9effaf84f5aaa1f33c3e2cf4ad5a,седьмой кассационный суд общей юрисдикции телефон,седьмой катоционный суд общей юрисдикции телефон,седьмой кассационный суд общей юрисдикции тел...,седьмой касационный суд общей юрисдикции телефон


In [8]:
def edit_distance(ref: Iterable, hyp: Iterable, plot: bool=False) -> int:
    dist = np.zeros((len(hyp) + 1, len(ref) + 1), dtype=np.int32)

    dist[:, 0] = np.arange(len(hyp) + 1)
    dist[0, :] = np.arange(len(ref) + 1)

    for i, r in enumerate(hyp, start=1):
        for j, h in enumerate(ref, start=1):
            dist[i, j] = min(
                dist[i - 1, j - 1] + (r != h),
                dist[i, j - 1] + 1,
                dist[i - 1, j] + 1
            )
    if plot:
        sns.heatmap(
            pd.DataFrame(
                dist,
                index=[' '] + list(hyp),
                columns=[' '] + list(ref)
            ),
            annot=True,
            cmap='coolwarm_r',
            linewidth=2
        )
        plt.tick_params(
            axis='both', which='major', labelsize=14, left=False, labelbottom=False,
            bottom=False, top=False, labeltop=True
        )
        plt.yticks(rotation=0)

    return dist[-1, -1]


In [9]:
def error_rate(refs: Iterable[Iterable], hyps: Iterable[Iterable]) -> float:
    """
    ignoring hypotheses with empty references
    """

    wrong_words, all_words = 0, 0

    for ref, hyp in tqdm(zip(refs, hyps), total=len(refs)):
        if len(ref) > 0:
            wrong_words += edit_distance(ref, hyp)
            all_words += len(ref)
        else:
            continue
    return wrong_words / all_words


def wer(refs: Iterable[str], hyps: Iterable[str]) -> float:
    """
    Word Error Rate
    """
    return error_rate(
        [ref.split() for ref in refs],
        [hyp.split() for hyp in hyps]
    )


def cer(refs: Iterable[str], hyps: Iterable[str]) -> float:
    """
    Character Error Rate
    """
    return error_rate(refs, hyps)

In [10]:
method2wer = {model: wer(val_df[model], val_df['text']) for model in MODEL_LIST}

  0%|          | 0/12354 [00:00<?, ?it/s]

  0%|          | 0/12354 [00:00<?, ?it/s]

  0%|          | 0/12354 [00:00<?, ?it/s]

In [11]:
method2wer

{'qnet': 0.7652783922138658,
 'w2v': 0.5601030720835285,
 'w2v_tts': 0.620017745017745}

In [12]:
def get_rover_df(df: pd.DataFrame, model_cols: List[str], tmp_col: str="__tmp") -> pd.DataFrame:

    rover_df = df.copy()

    if "text" in rover_df.columns:
        rover_df.drop("text", axis=1, inplace=True)

    rover_df[tmp_col] = rover_df.apply(lambda row: [(model, row[model]) for model in model_cols], axis=1)

    rover_df = rover_df.drop(model_cols, axis=1).explode(tmp_col)

    return pd.DataFrame({
        "task": rover_df["task"],
        "performer": rover_df[tmp_col].apply(lambda x: x[0]),
        "text": rover_df[tmp_col].apply(lambda x: x[1])
    })

In [13]:
val_rover_df = get_rover_df(val_df, model_cols=MODEL_LIST)

In [14]:
rover_result = (
    ROVER(
        tokenizer=lambda x: list(x),
        detokenizer=lambda s: "".join(s),
        silent=False
    )
        .fit_predict(val_rover_df)
)

  0%|          | 0/12354 [00:00<?, ?it/s]

In [15]:
rover_result

task
00126486c79ae5254e1ccff81bd06c52            баксето горске
0022ce6b98768587407ec6bafc726c3f           бориса глепском
00257e4f1cf7df02c33d658a06cd4d7c                 жуно гатх
00290854e17300262c423409acbc3694                    аргуна
002d20a21ceda7330dccb030fda53b8e     маршрут цытодель силы
                                             ...          
ffe3055f00e0b85ed591bd8b094b74da                    матола
ffed617e2f52afff8325ed67cea0e736       билет экспресс шина
ffedc21adf9f778a36e54e9a92296684                   аркадия
fff84841dfd4abcb56d1f646c56543ab    непски тридцать запять
fffa25bd1e4ca9152efe3558e389f68b     ближайший кот бигимот
Name: agg_text, Length: 12354, dtype: object

In [16]:
rover_result = pd.merge(
    val_df,
    rover_result.reset_index(),
    on='task'
)

In [17]:
method2wer['base_rover'] = wer(rover_result['agg_text'], rover_result['text'])
val_df = rover_result
MODEL_LIST.append("base_rover")

  0%|          | 0/12354 [00:00<?, ?it/s]

In [18]:
val_df = val_df.rename(columns={"agg_text":"base_rover"})

In [19]:
val_df

,task,text,qnet,w2v,w2v_tts,base_rover
0,3c2875271fb918da312865549d444653,антонине татариновой,онпанине то тариновый,антонине татариновый,антонини татариновый,антонине татариновый
1,1254c97d2434297a886e4dc4b2a16863,илюши власова,илюши влассво,и люши власово,и люше власова,и люши власово
2,867e45b9a914fffff0c8ef976ee0a275,терехину,тере гину,терехину,терьехину,терехину
3,32d052f80c431869e7c13914b88f958e,айдарчике,а и дарчики,аидарчике,а и дарчике,а и дарчике
4,719cbc316c0f3caa6324c110ad899b81,ксении валерьевны волковой,сени и воверивны в волколы,к сени и валеревны волковой,ксение ивалиревной волковой,ксени и валеревный волковой
...,...,...,...,...,...,...
12349,b2c4360bffc98f20b59fc6e6c3f1a606,фармэконом,фар мы коном,фар мконом,фар мыконон,фар мыконом
12350,de0fd646bdd3b37697580c2cc2d7d624,ссср бронь,этозысер бронь,эсососэр бронь,эсисыцар бронь,эсосысэр бронь
12351,97d2af838eb539e5a7151b099e13498d,фото салончик,хот то солоничиак,фот -то солончик,фото солоничек а,фот то солоничик
12352,f8df9effaf84f5aaa1f33c3e2cf4ad5a,седьмой кассационный суд общей юрисдикции телефон,седьмой катоционный суд общей юрисдикции телефон,седьмой кассационный суд общей юрисдикции тел...,седьмой касационный суд общей юрисдикции телефон,седьмой касационный суд общей юрисдикции телефон


In [20]:
def one_edit_words(word: str) -> Set[str]:
    """
    return list of candidates with one correction
    """
    letters = 'абвгдежзийклмнопрстуфхцчшщъыьэюя'
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletions = [left + right[1:] for left, right in splits if right]
    substitutions = [left + c + right[1:] for left, right in splits if right for c in letters]
    insertions = [left + c + right for left, right in splits for c in letters]
    return set(deletions + substitutions + insertions)


word_counts = Counter([word for utterance in train_df['text'].str.split() for word in utterance])


@lru_cache(maxsize=None)
def correct_word(word: str) -> str:
    if word in word_counts:
        return word

    candidates = one_edit_words(word)

    candidates = sorted([
        (word, word_counts[word])
        for word in candidates if word_counts[word] > 0
    ],
        key=lambda x: -x[1]
    )

    if candidates:
        return max(candidates, key=lambda x: x[1])[0]
    return word

In [21]:
base_rover_corrected = val_df['base_rover'].apply(
    lambda x: " ".join([correct_word(w) for w in x.split()])
)

method2wer['base_rover_corrected'] = wer(val_df['text'], base_rover_corrected)
val_df["base_rover_corrected"] = base_rover_corrected
MODEL_LIST.append("base_rover_corrected")


  0%|          | 0/12354 [00:00<?, ?it/s]

In [22]:
method2wer

{'qnet': 0.7652783922138658,
 'w2v': 0.5601030720835285,
 'w2v_tts': 0.620017745017745,
 'base_rover': 0.5634646316005613,
 'base_rover_corrected': 0.528392647633946}

In [23]:
w2v_corr = val_df['w2v'].apply(
    lambda x: " ".join([correct_word(w) for w in x.split()])
)

w2v_tts_corr = val_df['w2v_tts'].apply(
    lambda x: " ".join([correct_word(w) for w in x.split()])
)

wer(val_df['text'], w2v_corr)

corrected_val = pd.DataFrame()
corrected_val['w2v'] = w2v_corr
corrected_val['w2v_tts'] = w2v_tts_corr
corrected_val['qnet'] = val_df['qnet']
corrected_val['task'] = val_df['task']

  0%|          | 0/12354 [00:00<?, ?it/s]

In [24]:
corrected_val

,w2v,w2v_tts,qnet,task
0,антонина татариновый,антонина татариновый,онпанине то тариновый,3c2875271fb918da312865549d444653
1,и люди власов,и леше власов,илюши влассво,1254c97d2434297a886e4dc4b2a16863
2,терехину,терьехину,тере гину,867e45b9a914fffff0c8ef976ee0a275
3,аидарчике,а и дарчике,а и дарчики,32d052f80c431869e7c13914b88f958e
4,к сети и валерьевны волковой,ксения ивалиревной волковой,сени и воверивны в волколы,719cbc316c0f3caa6324c110ad899b81
...,...,...,...,...
12349,бар эконом,бар мыконон,фар мы коном,b2c4360bffc98f20b59fc6e6c3f1a606
12350,эсососэр бронь,эсисыцар бронь,этозысер бронь,de0fd646bdd3b37697580c2cc2d7d624
12351,фото сто солончик,фото солоничек а,хот то солоничиак,97d2af838eb539e5a7151b099e13498d
12352,седьмой кассационный суд общей юрисдикции телефон,седьмой касационный суд общей юрисдикции телефон,седьмой катоционный суд общей юрисдикции телефон,f8df9effaf84f5aaa1f33c3e2cf4ad5a


In [25]:
rover_on_corrected = get_rover_df(corrected_val, ['w2v', 'w2v_tts', 'qnet'])

In [26]:
rover_corrected_result = (
    ROVER(
        tokenizer=lambda x: list(x),
        detokenizer=lambda s: "".join(s),
        silent=False
    )
        .fit_predict(rover_on_corrected)
)

  0%|          | 0/12354 [00:00<?, ?it/s]

In [27]:
rover_corrected_result = pd.merge(
    val_df,
    rover_corrected_result.reset_index(),
    on='task'
)


In [28]:
method2wer['rover_on_corrected'] = wer(rover_corrected_result['agg_text'], val_df['text'])
MODEL_LIST.append("rover_on_corrected")


  0%|          | 0/12354 [00:00<?, ?it/s]

In [29]:
method2wer

{'qnet': 0.7652783922138658,
 'w2v': 0.5601030720835285,
 'w2v_tts': 0.620017745017745,
 'base_rover': 0.5634646316005613,
 'base_rover_corrected': 0.528392647633946,
 'rover_on_corrected': 0.48954751753541464}

In [30]:
class LaplaceLanguageModel:

    def __init__(
            self,
            tokenized_texts: Iterable[Iterable[str]],
            n: int,
            delta: float = 0.0,
            BOS: str='<BOS>',
            EOS: str='<EOS>'
    ):
        self.n = n
        self.BOS = BOS
        self.EOS = EOS
        ngram_counts: Dict[Tuple[str, ...], Dict[str, int]] = self.build_ngram_counts(
            tokenized_texts, n, BOS, EOS
        )

        self.vocab = {
            token for distribution in ngram_counts.values() for token in distribution
        }

        self.probs = defaultdict(Counter)

        for prefix, distribution in ngram_counts.items():
            norm: float = sum(distribution.values()) + delta * len(self.vocab)
            self.probs[prefix] = {
                token: (count + delta) / norm for token, count in distribution.items()
            }

    @staticmethod
    def build_ngram_counts(
            tokenized_texts: Iterable[Iterable[str]],
            n: int,
            BOS: str,
            EOS: str
    ) -> Dict[Tuple[str, ...], Dict[str, int]]:

        counts = defaultdict(Counter)

        for text in tokenized_texts:

            ngrams = nltk.ngrams(
                text, n=n, pad_left=True, pad_right=True, left_pad_symbol=BOS, right_pad_symbol=EOS
            )

            for ngram in ngrams:
                prev, token = ngram[:-1], ngram[-1]
                counts[prev][token] += 1

        return counts


    def __get_observed_token_distribution(self, prefix: List[str]) -> Dict[str, float]:
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [self.BOS] * (self.n - 1 - len(prefix)) + prefix
        return self.probs[tuple(prefix)]


    def get_token_distribution(self, prefix: List[str]) -> Dict[str, float]:

        distribution: Dict[str, float] = self.__get_observed_token_distribution(prefix)

        missing_prob_total: float = 1.0 - sum(distribution.values())

        missing_prob = missing_prob_total / max(1, len(self.vocab) - len(distribution))

        return {token: distribution.get(token, missing_prob) for token in self.vocab}


    def get_next_token_prob(self, prefix: List[str], next_token: str):

        distribution: Dict[str, float] = self.__get_observed_token_distribution(prefix)

        if next_token in distribution:
            return distribution[next_token]

        else:
            missing_prob_total = 1.0 - sum(distribution.values())
            return max(0, missing_prob_total) / max(1, len(self.vocab) - len(distribution))


    def score_sequence(self, tokens: List[str], min_logprob: float = np.log(10 ** -50.)) -> float:
        prefix = [self.BOS] * (self.n - 1)
        padded_tokens = tokens + [self.EOS]
        logprobs_sum = 0.0
        for token in padded_tokens:
            logprob = np.log(self.get_next_token_prob(prefix, token))
            prefix = prefix[1:] + [token]
            logprobs_sum += max(logprob, min_logprob)
        return logprobs_sum / len(tokens) if tokens else 0.0

In [31]:
text_data_np = np.array(text_data)
lm = LaplaceLanguageModel(
    n=2,
    tokenized_texts=text_data_np[0],
    delta=1e-5
)

In [32]:
for text in ('мама мыла раму', 'мамо мыла раму', 'машинное обучение', 'маинное обучение'):
    score = lm.score_sequence(list(text))
    print(f"{text}\t\t{score:.2f}")

мама мыла раму		-5.46
мамо мыла раму		-4.70
машинное обучение		-2.91
маинное обучение		-2.99


In [33]:
max_likelihood_utterances = val_df.apply(
    lambda row: row[
        np.array([
            lm.score_sequence(tokens=list(row[model])) for model in MODEL_LIST
        ]).argmax()
    ],
    axis=1
)

KeyError: 'rover_on_corrected'

In [ ]:
method2wer['dummy_rescoring'] = wer(val_df['text'], max_likelihood_utterances)

In [ ]:
def get_best_transcription(ref: Iterable[str], hyps: Iterable[Iterable[str]]):
    return hyps[
        np.array([
            edit_distance(ref, hyp) for hyp in hyps
        ]).argmin()
    ]

In [ ]:
oracle_hyp = val_df.apply(
    lambda row: " ".join(
        get_best_transcription(
            ref=row['text'].split(),
            hyps=[row[model].split() for model in MODEL_LIST]
        )
    ),
    axis=1
)

In [ ]:
method2wer['oracle_wer'] = wer(val_df['text'], oracle_hyp)

In [ ]:
method2wer

In [34]:
#ИТОГ
w2v_corr = test_df['w2v'].apply(
    lambda x: " ".join([correct_word(w) for w in x.split()])
)

w2v_tts_corr = test_df['w2v_tts'].apply(
    lambda x: " ".join([correct_word(w) for w in x.split()])
)

corrected_test = pd.DataFrame()
corrected_test['w2v'] = w2v_corr
corrected_test['w2v_tts'] = w2v_tts_corr
corrected_test['qnet'] = test_df['qnet']
corrected_test['task'] = test_df['task']

In [35]:
rover_test_df = get_rover_df(corrected_test, ['w2v', 'w2v_tts', 'qnet'])

In [36]:
rover_test = (
    ROVER(
        tokenizer=lambda x: list(x),
        detokenizer=lambda s: "".join(s),
        silent=False
    )
        .fit_predict(rover_test_df)
)

  0%|          | 0/18875 [00:00<?, ?it/s]

In [37]:
rover_test_merge = pd.merge(
    test_df,
    rover_test.reset_index(),
    on='task'
)

In [38]:
rover_test_merge = rover_test_merge.rename({'agg_text': 'prediction'}, axis=1)
rover_test_merge

,task,qnet,w2v,w2v_tts,prediction
0,73aff8bc8d99278c6ca6d1ac243557e3,дели,зили,зили,лили
1,ba8443f3cc91e56667278db36dea02b7,вектор упавнович чунула к лещенуму,виктору павловичу новокрещеному,виктору павлновичу навокрещинову оо,виктори павлновичу навокрещеному
2,af346fae1d5686a357e32710c5d4f13e,риме менниковой,римми мельниковой,рымми мельниковой,римми мельниковой
3,262fda7ab40a8417b99ecb314b3f7405,александровик провичо,александру викторовичу,александру викторовичу,александру викторовича
4,08bd8fb35ceaf2843734ffeb389f2283,я ро славцов,ерославцев,ярославцев,ярославцев
...,...,...,...,...,...
18870,35e142d32571927fa08227a5d1a89152,пермский государственный институт культуры,пермский государственный институт культуры,пярмский государственный институт культуры,пермский государственный институт культуры
18871,0c7beaae1712adf488fae74a4966b661,дальфурму на карте,даль фарма на карте,дальфарман на карте,дальфарма на карте
18872,5de895309c234ced55d31f001e930250,городская полекленника но меродинадцать,городская полеклиника номер одиннадцать,городцкое полеклене эко номер одинадцать,городская полекленника номер одиннадцать
18873,beda99f2bada550facb9a70c1c48a62b,билет шафран,билет шафран,белет шафран,билет шафран


In [51]:
test_result = rover_test_merge[["task", "prediction"]]

In [52]:
test_result

,task,prediction
0,73aff8bc8d99278c6ca6d1ac243557e3,лили
1,ba8443f3cc91e56667278db36dea02b7,виктори павлновичу навокрещеному
2,af346fae1d5686a357e32710c5d4f13e,римми мельниковой
3,262fda7ab40a8417b99ecb314b3f7405,александру викторовича
4,08bd8fb35ceaf2843734ffeb389f2283,ярославцев
...,...,...
18870,35e142d32571927fa08227a5d1a89152,пермский государственный институт культуры
18871,0c7beaae1712adf488fae74a4966b661,дальфарма на карте
18872,5de895309c234ced55d31f001e930250,городская полекленника номер одиннадцать
18873,beda99f2bada550facb9a70c1c48a62b,билет шафран


In [41]:
test_df

,task,qnet,w2v,w2v_tts
0,73aff8bc8d99278c6ca6d1ac243557e3,дели,зили,зили
1,ba8443f3cc91e56667278db36dea02b7,вектор упавнович чунула к лещенуму,виктору павловичу новокрещеному,виктору павлновичу навокрещинову оо
2,af346fae1d5686a357e32710c5d4f13e,риме менниковой,римми мельниковой,рымми мельниковой
3,262fda7ab40a8417b99ecb314b3f7405,александровик провичо,александру викторовичу,александру викторовичу
4,08bd8fb35ceaf2843734ffeb389f2283,я ро славцов,ерославцев,ярославцев
...,...,...,...,...
18870,35e142d32571927fa08227a5d1a89152,пермский государственный институт культуры,пермский государственный институт культуры,пярмский государственный институт культуры
18871,0c7beaae1712adf488fae74a4966b661,дальфурму на карте,даль фарма на карте,дальфарман на карте
18872,5de895309c234ced55d31f001e930250,городская полекленника но меродинадцать,городская полеклиника номер одиннадцать,городцкое полеклене эко номер одинадцать
18873,beda99f2bada550facb9a70c1c48a62b,билет шафран,билет шафран,белет шафран


In [54]:
username = "maratgasanov"

test_result.to_json(
    f"noisy_text_aggregation_test_prediction_{username}.jsonl",
    lines=True, orient="records"
)